In [1]:
from IPython.core.display import display, HTML, Markdown
from ipywidgets import Button, HBox, VBox, widgets, Layout
from tabulate import tabulate
import pandas as pd

In [16]:
#solver solo per non fare i calcoli a mano

def num_percorsi(env,r,c):
    m = []
    for i in range(0,r):
        m.append([0 for j in range(0,c)])
    hole = 0
    i=0
    j=0
    for i in range(0,r):
        m[i][0]=0
    for j in range(0,c):
        m[0][j]=0
    for i in range(0,r):
        for j in range(0,c):
            if i==1 and j==1:
                m[i][j]=1
            else:
                if str(env[i][j])=="X":
                    m[i][j]="X"
                else:
                    if m[i-1][j] == "X" and m[i][j-1] == "X":
                         m[i][j] =0                    
                    elif m[i-1][j] == "X":
                        m[i][j]= m[i][j-1];
                    elif m[i][j-1] == "X":
                        m[i][j]= m[i-1][j]
                    else:
                        m[i][j] = m[i-1][j] + m[i][j-1];
        
    return m;

def num_percorsi_reverse(env,r,c):
    m = []
    for i in range(0,r):
        m.append([0 for j in range(0,c)])
    hole = 0
    i=r-1
    j=c-1
    while i > 0:
        if str(env[i][c-1])=="X":
            hole=1
            m[i][c-1]="X"
        elif hole == 0:
            m[i][c-1] = 1
        else:
            m[i][c-1] = 0
        i-=1
    hole=0
    while j > 0:
        if str(env[r-1][j])=="X":
            hole=1
            m[r-1][j] = "X"
        elif hole == 0:
            m[r-1][j] = 1
        else:
            m[r-1][j] = 0
        j-=1
    for i in range(r-2,0,-1):
        for j in range(c-2,0,-1):
                if str(env[i][j])=="X":
                    m[i][j]="X"
                else:
                    if m[i+1][j] == "X" and m[i][j+1] == "X":
                         m[i][j] =0                    
                    elif m[i+1][j] == "X":
                        m[i][j]= m[i][j+1]
                    elif m[i][j+1] == "X":
                        m[i][j]= m[i+1][j]
                    else:
                        #print(f"{m[i+1][j]}+{m[i][j+1]}")
                        m[i][j] = m[i+1][j] + m[i][j+1];
    return m
            


In [93]:
def evaluation_format(answ, pt_green,pt_red):
    pt_blue=0
    if pt_green!=0:
        pt_blue=pt_red-pt_green
        pt_red=0
    return f"{answ}. Totalizzeresti <span style='color:green'>[{pt_green} safe pt]</span>, \
                                    <span style='color:blue'>[{pt_blue} possible pt]</span>, \
                                    <span style='color:red'>[{pt_red} out of reach pt]</span>.<br>"

def check_num_paths_to_goal(num_path, row, col):
    for i in range (0,row-1):
        if num_path[i][0]!=0:
            return evaluation_format("No", 0, 10)+"Attenzione, il numero di cammini parte dalla cella $(1,1)$"
    for j in range (0,col-1):
        if num_path[0][j]!=0:
            return evaluation_format("No", 0, 10)+"Attenzione, il numero di cammini parte dalla cella $(1,1)$"
    for i in range(row-1,0,-1):
        for j in range (col-1,0,-1):
            if num_path[i][j]!="X":
                if i==1 and j==1:
                    if num_path[i][j]!=1:
                        return  evaluation_format("No", 0, 10)+"Il valore nella cella $(1,1)$ non è corretto"
                elif num_path[i-1][j]!="X" and num_path[i][j-1]!="X":
                    if num_path[i][j]!=num_path[i-1][j]+num_path[i][j-1]:
                        return  evaluation_format("No", 0, 10)+"La tua soluzione non è coerente"
                elif num_path[i-1][j]!="X":
                    if num_path[i][j]!=num_path[i-1][j]:
                        return  evaluation_format("No", 0, 10)+"La tua soluzione non è coerente"
                elif num_path[i][j-1]!="X":
                    if num_path[i][j]!=num_path[i][j-1]:
                        return  evaluation_format("No", 0, 10)+"La tua soluzione non è coerente"
                else:
                    if num_path[i][j]!=0:
                        return  evaluation_format("No", 0, 10)+"La tua soluzione non è coerente"
    return  evaluation_format("Si", 1, 10)+"La tua soluzione è coerente"

def check_num_paths_to_start(num_path, row, col):
    for i in range(1,row-1):
        for j in range (1,col-1):
            if num_path[i][j]!="X":
                if i==row-1 and j==col-1:
                    if num_path[i][j]!=1:
                        return  evaluation_format("No", 0, 10)+"Il valore nella cella $(n+1,m+1)$ non è corretto"
                elif num_path[i+1][j]!="X" and num_path[i][j+1]!="X":
                    if num_path[i][j]!=num_path[i+1][j]+num_path[i][j+1]:
                        return  evaluation_format("No", 0, 10)+"La tua soluzione non è coerente"
                elif num_path[i+1][j]!="X":
                    if num_path[i][j]!=num_path[i+1][j]:
                        return  evaluation_format("No", 0, 10)+"La tua soluzione non è coerente"
                elif num_path[i][j+1]!="X":
                    if num_path[i][j]!=num_path[i][j+1]:
                        return  evaluation_format("No", 0, 10)+"La tua soluzione non è coerente"
                else:
                    if num_path[i][j]!=0:
                        return  evaluation_format("No", 0, 10)+"La tua soluzione non è coerente"
    return  evaluation_format("Si", 1, 10)+"La tua soluzione è coerente"
                
def check_value(name,table,row,col, value):
    submission=f"Hai inserito il valore {value}, nella tabella {name} in posizione ({row},{col}) è presente il valore {table[row][col]}"
    if table[row][col]==value:
        return evaluation_format("Si", 1, 10)+f"La tua soluzione è coerente con quanto hai inserito nella tabella {name}<br>"+submission
    else:
        return evaluation_format("No", 0, 10)+f"La tua soluzione è coerente con quanto hai inserito nella tabella {name}<br>"+submission
    
def check_value_from_tables(table1, table2, row, col, value):
    submission=f"Hai inserito il valore {value}, il valore atteso ricavato dalle tue tabelle è {table1[row][col]*table2[row][col]}"
    if table[row][col]==value:
        return evaluation_format("Si", 1, 10)+f"La tua soluzione è coerente con quanto hai inserito nelle tabelle<br>"+submission
    else:
        return evaluation_format("No", 0, 10)+f"La tua soluzione è coerente con quanto hai inserito nella tabelle<br>"+submission

## Esercizio \[60 pts\]
(campo minato) Ricerca dei cammini.

Un robot R, inizialmente situato nella cella $A1$, deve portarsi nella sua home H situata nella cella $H9$.

In [54]:
table = [
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, "R" , 2 , 3 , 1 , 1 , 1 , 0 , 0 , "X" ],
         [0, 3 , 3 , 1 , 0 , "X" , "X" , 0 , 0 , 0 ],
         [0, 2 , "X" , 0 , "X" , 0 , 0 , 1 , 1 , 1 ],
         [0, 0 , 0 , 1 , 0 , 0 , 0 , 1 , "X" , 0 ],
         [0, 0 , 0 , "X" , 1 , "X" , 1 , 0 , 0 , 0 ],
         [0, 0 , 1 , 1 , 1 , 0 , 3 , "X" , 0 , 1 ],
         [0, 3 , "X" , 0 , 1 , 2 , 0 , 0 , 1 , 0 ],
         [0, 2 , 1 , 2 , 1 , 2 , 1 , 2 , 1 , "H"] ,
        ]


columns=["1","2","3","4","5","6","7","8","9"]
index=pd.Index(["0","A","B","C","D","E","F","G","H"])
df=pd.DataFrame(table,index=index)
df = df.iloc[1:,1:]



print(tabulate(df, headers=columns, tablefmt='fancy_grid'))
#display(HTML(df.to_html()))

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │ R   │ 2   │ 3   │ 1   │ 1   │ 1   │ 0   │ 0   │ X   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │ 3   │ 3   │ 1   │ 0   │ X   │ X   │ 0   │ 0   │ 0   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │ 2   │ X   │ 0   │ X   │ 0   │ 0   │ 1   │ 1   │ 1   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │ 0   │ 0   │ 1   │ 0   │ 0   │ 0   │ 1   │ X   │ 0   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │ 0   │ 0   │ X   │ 1   │ X   │ 1   │ 0   │ 0   │ 0   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │ 0   │ 1   │ 1   │ 1   │ 0   │ 3   │ X   │ 0   │ 1   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │ 3   │ X   │ 0   │ 1   │ 2   │ 0   │ 0   │ 1   │ 0   │
├────┼─────┼─────┼─────┼

Ogni cella senza mina contiene un numero di monetine come rappresentato.\
Quante monetine può raccogliere al massimo il robot?

I movimenti base possibili sono il passo verso destra (ad esempio dalla cella A4 alla cella A5) ed il passo verso in basso (ad esempio dalla cella A4 alla cella B4).\
Tuttavia il robot non può visitare le celle occupate da un Pacman (indicate con la X).

__Richieste__:

1. __\[10 pts\]__ Costrisci una matrice *num_paths_to_goal* di dimensione $(m+1)x(n+1)$ dove per ogni $i = 0,..., m$ e $j = 0,..., n$ si riporti il numero di cammini dalla cella $(1,1)$ alla cella $(i,j)$

In [55]:
#num_paths_to_goal=[
#    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
#    [0, 1, 1, 1, 1, 1, 1, 1, 1, 'X'], 
#    [0, 1, 2, 3, 4, 'X', 'X', 1, 2, 2], 
#    [0, 1, 'X', 3, 'X', 0, 0, 1, 3, 5], 
#    [0, 1, 1, 4, 4, 4, 4, 5, 'X', 5], 
#    [0, 1, 2, 'X', 4, 'X', 4, 9, 9, 14], 
#    [0, 1, 3, 3, 7, 7, 11, 'X', 9, 23], 
#    [0, 1, 'X', 3, 10, 17, 28, 28, 37, 60], 
#    [0, 1, 1, 4, 14, 31, 59, 87, 124, 184]
#]
num_paths_to_goal= num_percorsi(table,len(table),len(table[0]))

In [56]:
print("La soluzione proposta è la seguente:")
df2=pd.DataFrame(num_paths_to_goal,index=index)
df2 = df2.iloc[1:,1:]
print(tabulate(df2, headers=columns, tablefmt='fancy_grid'))

La soluzione proposta è la seguente:
╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │   1 │ 1   │ 1   │ 1   │ 1   │ 1   │ 1   │ 1   │ X   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │   1 │ 2   │ 3   │ 4   │ X   │ X   │ 1   │ 2   │ 2   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │   1 │ X   │ 3   │ X   │ 0   │ 0   │ 1   │ 3   │ 5   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │   1 │ 1   │ 4   │ 4   │ 4   │ 4   │ 5   │ X   │ 5   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │   1 │ 2   │ X   │ 4   │ X   │ 4   │ 9   │ 9   │ 14  │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │   1 │ 3   │ 3   │ 7   │ 7   │ 11  │ X   │ 9   │ 23  │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │   1 │ X   │ 3   │ 10  │ 17  │ 28  │ 28  │

In [59]:
display(Markdown(check_num_paths_to_goal(num_paths_to_goal,len(num_paths_to_goal),len(num_paths_to_goal[0]))))

Si. Totalizzeresti <span style='color:green'>[1 safe pt]</span>,                                     <span style='color:blue'>[9 possible pt]</span>,                                     <span style='color:red'>[0 out of reach pt]</span>.<br>La tua soluzione è coerente

1. __\[10 pts\]__ Costrisci una matrice *num_paths_to_start* di dimensione $(m+1)x(n+1)$ dove per ogni $i = 0,..., m$ e $j = 0,..., n$ si riporti il numero di cammini dalla cella $(i,j)$ alla cella $(1,1)$

In [60]:
num_paths_to_start=[
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                        [0, 184, 63, 36, 9, 9, 9, 9, 2, 'X'], 
                        [0, 121, 27, 27, 0, 'X', 'X', 7, 2, 1], 
                        [0, 94, 'X', 27, 'X', 29, 17, 5, 1, 1], 
                        [0, 94, 49, 27, 27, 12, 12, 4, 'X', 1], 
                        [0, 45, 22, 'X', 15, 'X', 8, 4, 4, 1], 
                        [0, 23, 22, 22, 15, 9, 4, 'X', 3, 1], 
                        [0, 1, 'X', 7, 6, 5, 4, 3, 2, 1], 
                        [0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
                    ]
#num_paths_to_start=num_percorsi_reverse(table,len(table),len(table[0]))

In [61]:

df3=pd.DataFrame(num_paths_to_start,index=index)
df3 = df3.iloc[1:,1:]
print(tabulate(df3, headers=columns, tablefmt='fancy_grid'))

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │ 184 │ 63  │ 36  │ 9   │ 9   │ 9   │ 9   │ 2   │ X   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │ 121 │ 27  │ 27  │ 0   │ X   │ X   │ 7   │ 2   │ 1   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │  94 │ X   │ 27  │ X   │ 29  │ 17  │ 5   │ 1   │ 1   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │  94 │ 49  │ 27  │ 27  │ 12  │ 12  │ 4   │ X   │ 1   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │  45 │ 22  │ X   │ 15  │ X   │ 8   │ 4   │ 4   │ 1   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │  23 │ 22  │ 22  │ 15  │ 9   │ 4   │ X   │ 3   │ 1   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │   1 │ X   │ 7   │ 6   │ 5   │ 4   │ 3   │ 2   │ 1   │
├────┼─────┼─────┼─────┼

In [62]:
display(Markdown(check_num_paths_to_start(num_paths_to_start,len(num_paths_to_start),len(num_paths_to_start[0]))))

Si. Totalizzeresti <span style='color:green'>[1 safe pt]</span>,                                     <span style='color:blue'>[9 possible pt]</span>,                                     <span style='color:red'>[0 out of reach pt]</span>.<br>La tua soluzione è coerente

1. __\[10 pts\]__ Quanti sono i percorsi possibili se la partenza è in A–1?

In [79]:
ans1 = 184

In [80]:
display(Markdown(check_value("num_paths_to_start",num_paths_to_start,1,1,ans1)))

Si. Totalizzeresti <span style='color:green'>[1 safe pt]</span>,                                     <span style='color:blue'>[9 possible pt]</span>,                                     <span style='color:red'>[0 out of reach pt]</span>.<br>La tua soluzione è coerente con quanto hai inserito nella tabella num_paths_to_start<br>Hai inserito il valore 184, nella tabella num_paths_to_start in posizione (1,1) è presente il valore 184

2. __\[10 pts\]__ E se la partenza è in B–3?

In [82]:
ans2=27

In [83]:
display(Markdown(check_value("num_paths_to_start",num_paths_to_start,2,3,ans2)))

Si. Totalizzeresti <span style='color:green'>[1 safe pt]</span>,                                     <span style='color:blue'>[9 possible pt]</span>,                                     <span style='color:red'>[0 out of reach pt]</span>.<br>La tua soluzione è coerente con quanto hai inserito nella tabella num_paths_to_start<br>Hai inserito il valore 27, nella tabella num_paths_to_start in posizione (2,3) è presente il valore 27

3. __\[10 pts\]__ E se con partenza in A–1 il robot deve giungere in F–6?

In [88]:
ans3=11

In [89]:
display(Markdown(check_value("num_paths_to_goal",num_paths_to_goal,6,6,ans3)))

Si. Totalizzeresti <span style='color:green'>[1 safe pt]</span>,                                     <span style='color:blue'>[9 possible pt]</span>,                                     <span style='color:red'>[0 out of reach pt]</span>.<br>La tua soluzione è coerente con quanto hai inserito nella tabella num_paths_to_goal<br>Hai inserito il valore 11, nella tabella num_paths_to_goal in posizione (6,6) è presente il valore 11

4. __\[10 pts\]__ E se con partenza in A–1 ed arrivo in H–9 al robot viene richiesto di passare per
la cella D–5?

In [94]:
ans4=0

In [95]:
display(Markdown(check_value_from_tables(num_paths_to_start,num_paths_to_goal,4,5,ans4)))

Si. Totalizzeresti <span style='color:green'>[1 safe pt]</span>,                                     <span style='color:blue'>[9 possible pt]</span>,                                     <span style='color:red'>[0 out of reach pt]</span>.<br>La tua soluzione è coerente con quanto hai inserito nelle tabelle<br>Hai inserito il valore 0, il valore atteso ricavato dalle tue tabelle è 48

5. __\[10 pts\]__ Quale é il massimo valore in premi raccoglibili lungo una traversata da A–1 a H–9?

6. __\[10 pts\]__ Quanti sono i percorsi possibili che assicurino di portare a case tale massimo valore?